In [12]:
import pandas as pd
import random
import fastavro
import time

In [13]:
# Valores possíveis por coluna
nomes = ["Pedro", "Tiago", "João", "André", "Filipe", "Judas Iscariotes", "Mateus", "Tomé", "Bartolomeu", "Judas Tadeu", "Simão"]
idades = [20, 23, 19, 30, 28, 25, 31, 36, 34, 27, 39, 41, 52]
cores_favoritas = ["Vermelho", "Azul", "Verde", "Amarelo", "Roxo", "Laranja", "Anil"] 
animais_favoritos = ["Cachorro", "Gato", "Pássaro", "Peixe", "Coelho", "Bode", "Dragão de Komodo"]  # Adicione mais valores, se desejar
times_que_torce = ["Flamengo", "Corinthians", "Palmeiras", "São Paulo", "Grêmio", "Vasco", "Belo", "Campinense", "Treze"]  # Adicione mais valores, se desejar
estados_que_nasceu = ["Paraíba", "Acre", "São Paulo", "Rio de Janeiro", "Minas Gerais", "Bahia", "Pernambuco"]  # Adicione mais valores, se desejar

# Criando uma lista de dicionários com valores aleatórios usando random
dados_aleatorios = []
for i in range(1000000):
    dados_aleatorios.append({
        "nome": random.choice(nomes),
        "idade": random.choice(idades),
        "número favorito": random.randint(0, 1000),
        "cor favorita": random.choice(cores_favoritas),
        "animal favorito": random.choice(animais_favoritos),
        "time que torce": random.choice(times_que_torce),
        "estado que nasceu": random.choice(estados_que_nasceu)
    })

df = pd.DataFrame(dados_aleatorios)


In [14]:
# Definindo o esquema Avro
# obs: Lembrar de colocar null pra não ter problema com celulas vazias de uma base baixda da internet
schema = {
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "nome", "type": "string"},
        {"name": "idade", "type": ["int", "null"]},
        {"name": "número favorito", "type": ["int", "null"]},
        {"name": "cor favorita", "type": ["string", "null"]},
        {"name": "animal favorito", "type": ["string", "null"]},
        {"name": "time que torce", "type": ["string", "null"]},
        {"name": "estado que nasceu", "type": ["string", "null"]}
    ]
}

# Salvando em avro
with open("dados.avro", "wb") as avro_file:
    fastavro.writer(avro_file, schema, df.to_dict(orient="records"))


In [15]:
# Salvando em csv
df.to_csv("dados.csv", index=False)

In [16]:
# DataFrame de exemplo
data = {"col1": range(10000000), "col2": [str(i) for i in range(10000000)]}
df = pd.DataFrame(data)

# Medindo o tempo de escrita em CSV
start_time_csv_write = time.time()
df.to_csv("dados.csv", index=False)
csv_write_time = time.time() - start_time_csv_write

# Medindo o tempo de escrita em Avro
schema = {
    "type": "record",
    "name": "MyData",
    "fields": [
        {"name": "col1", "type": "int"},
        {"name": "col2", "type": "string"}
    ]
}

start_time_avro_write = time.time()
with open("dados.avro", "wb") as avro_file:
    fastavro.writer(avro_file, schema, df.to_dict(orient="records"))
avro_write_time = time.time() - start_time_avro_write

print(f"Tempo de escrita em CSV: {csv_write_time:.4f} segundos")
print(f"Tempo de escrita em Avro: {avro_write_time:.4f} segundos")


Tempo de escrita em CSV: 12.9384 segundos
Tempo de escrita em Avro: 30.9859 segundos


In [17]:
# Medindo o tempo de leitura em CSV
start_time_csv_read = time.time()
df_csv = pd.read_csv("dados.csv")
csv_read_time = time.time() - start_time_csv_read

# Medindo o tempo de leitura em Avro
start_time_avro_read = time.time()
with open("dados.avro", "rb") as avro_file:
    avro_reader = fastavro.reader(avro_file)
    avro_records = list(avro_reader)
df_avro = pd.DataFrame(avro_records)
avro_read_time = time.time() - start_time_avro_read

print(f"Tempo de leitura em CSV: {csv_read_time:.4f} segundos")
print(f"Tempo de leitura em Avro: {avro_read_time:.4f} segundos")


Tempo de leitura em CSV: 2.1861 segundos
Tempo de leitura em Avro: 21.8920 segundos
